To Do List:

- Additional Principal Payment Calculations
- Mortgage Insurance Calculations
- Rent Growth Calculations


In [1]:
# Dependencies and Setup
import json
import matplotlib.pyplot as plt
import pandas as pd
from functools import reduce
import numpy as np
import requests
import datetime as dt
from datetime import datetime, date, time, timedelta
from dateutil.relativedelta import relativedelta
import math
import scipy.stats as stats
from scipy.stats import linregress
from splinter import Browser
from bs4 import BeautifulSoup
import pymongo
from flask import Flask, render_template, redirect
from flask import Flask, jsonify, render_template
from flask_pymongo import PyMongo

In [2]:

# street = input("Steet: ")
# city = input("City: ")
# state = input("State: ")
# zip_code = input("Zip Code: ")

In [3]:
# Property Details (Input All)
street = "123 Main St"
city = "Plainsboro"
state = "New Jersey"
zip_code = "08540"

In [4]:
# Purchase Details (Input:- Purchase Price, HOA Fees, Property Taxes, Homeowner's Insurance, Purchase Date)
# Purchase Price:- NJ Average Sales Price (NJ Division of Taxation)
purchase_price = 412284
hoa_fees = 0
# Property Taxes Estimate (2% of Home Value)
property_taxes = purchase_price * .02
property_tax_monthly = round((property_taxes / 12), 2)
property_tax_rate = property_taxes / purchase_price
homeowners_insurance_annual = 900
homeowners_insurance_monthly = homeowners_insurance_annual / 12
#today = date.today()
#purchase_date = today.strftime("%Y-%m-%d")
purchase_date = datetime.strptime("2019-03-29", "%Y-%m-%d")
month_start = datetime(purchase_date.year, purchase_date.month + 2, 1)
property_tax_monthly

687.14

In [5]:
# Borrower Profile (Input All)
current_address = "321 Niam Ave, East Brunswick, NJ 08340"
own_or_rent = "Rent"
current_housing_payment = 2200
credit_score = 720


In [6]:
# Loan Details (Input Loan Type, LTV, Interest Rate, Term, Additional Principal)
loan_type = "Conventional"
ltv = 0.80
downpayment = round((purchase_price * (1 - ltv)), 2)
loan_amount = round((purchase_price * ltv), 2)
interest_rate = 3.75 / 100
term_annual = 30
no_of_months = term_annual * 12
monthly_p_i = round((-1 * np.pmt(interest_rate / 12, no_of_months, loan_amount)), 2)
additional_principal = 0

pitia = monthly_p_i + property_tax_monthly + homeowners_insurance_monthly + homeowners_insurance_monthly
pitia


2364.62

In [7]:
loan_amount

329827.2

In [8]:
# Mortgage Insurance Details



In [9]:
### Closing Costs (Input Total Loan Costs, Other Costs, Lender Credits, Seller Credits)
# Total Loan Costs (A + B + C)
total_loan_costs = 5000.00
# Other Costs (E + F + G + H)
other_costs = 5000.00
# Prepaids (3 Months)
prepaids = round((3 * (property_tax_monthly + homeowners_insurance_monthly)), 2)
# Escrows (3 Months)
escrows = round((3 * (property_tax_monthly + homeowners_insurance_monthly)), 2)
lender_credits = - 0
seller_credits = - 0
total_closing_costs = round((total_loan_costs + other_costs + prepaids + escrows + lender_credits), 2)
cash_to_close = downpayment + total_closing_costs
going_in_costs = total_loan_costs + other_costs + lender_credits + seller_credits

going_in_costs



10000.0

In [10]:
# Appreciation Forecast
appreciation_rate = 3.425


In [11]:
# Resale Scenario (Input All)

agent_commission = 5 / 100

# 10-Yr Breakeven Inflation can be used
# 2% used here
inflation = 2.0 / 100


In [12]:
# Period to calculate
per = 1

# Calculate the interest payment
ipmt = np.ipmt(interest_rate / 12, per, no_of_months, loan_amount)

# Calculate the principal payment
ppmt = np.ppmt(interest_rate / 12, per, no_of_months, loan_amount)

print(ipmt, ppmt)


-1030.71 -496.7711884457733


In [13]:
# build a DateTimeIndex Range for the next 30 years based on MS (Month Start)
rng = pd.date_range(month_start, periods=term_annual * 12, freq='MS')
rng.name = "payment_date"
rng

DatetimeIndex(['2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01',
               '2019-09-01', '2019-10-01', '2019-11-01', '2019-12-01',
               '2020-01-01', '2020-02-01',
               ...
               '2048-07-01', '2048-08-01', '2048-09-01', '2048-10-01',
               '2048-11-01', '2048-12-01', '2049-01-01', '2049-02-01',
               '2049-03-01', '2049-04-01'],
              dtype='datetime64[ns]', name='payment_date', length=360, freq='MS')

In [14]:
# Create Amortization Schedule DataFrame
amortization_schedule_df = pd.DataFrame()

dates = []
for d in rng:
    date1 = np.datetime64(d)
    date2 = np.datetime_as_string(date1, unit='D')
    dates.append(date2)    
    
amortization_schedule_df["payment_date"] = dates
amortization_schedule_df["month"] = list(range(1, (len(amortization_schedule_df["payment_date"]) + 1)))

#Year
periods = list(range(1, (len(amortization_schedule_df["payment_date"]) + 1)))
year = []
for x in periods: 
    year.append(math.ceil( x / 12 ))
    
# Year Column 
amortization_schedule_df["year"] = year

# Payment, Principal, Interest
amortization_schedule_df["payment"] = np.pmt(interest_rate/12, term_annual*12, loan_amount)
amortization_schedule_df["principal"] = np.ppmt(interest_rate/12, amortization_schedule_df.index, term_annual*12, loan_amount)
amortization_schedule_df["interest"] = np.ipmt(interest_rate/12, amortization_schedule_df.index, term_annual*12, loan_amount)

# Additional Principal. Convert to a negative value in order to keep the signs the same
amortization_schedule_df["additional_principal"] = -additional_principal

# Cumulative Principal, Cumulative Interest
amortization_schedule_df["cumulative_principal"] = (amortization_schedule_df["principal"] + amortization_schedule_df["additional_principal"]).cumsum()
amortization_schedule_df["cumulative_interest"] = (amortization_schedule_df["interest"]).cumsum()

# Ending Balance at end of period
amortization_schedule_df["ending_balance"] = loan_amount + amortization_schedule_df["cumulative_principal"]

# Loan-To-Value
amortization_schedule_df["loan_to_value_%"] = (amortization_schedule_df["ending_balance"] / purchase_price) * 100
# Round number to 2 decimal points
amortization_schedule_df = amortization_schedule_df.round(2)


# Preview DataFrame
amortization_schedule_df


,payment_date,month,year,payment,principal,interest,additional_principal,cumulative_principal,cumulative_interest,ending_balance,loan_to_value_%
0,2019-05-01,1,1,-1527.48,-495.22,-1032.26,0,-495.22,-1032.26,329331.98,79.88
1,2019-06-01,2,1,-1527.48,-496.77,-1030.71,0,-991.99,-2062.97,328835.21,79.76
2,2019-07-01,3,1,-1527.48,-498.32,-1029.16,0,-1490.32,-3092.13,328336.88,79.64
3,2019-08-01,4,1,-1527.48,-499.88,-1027.60,0,-1990.20,-4119.73,327837.00,79.52
4,2019-09-01,5,1,-1527.48,-501.44,-1026.04,0,-2491.64,-5145.76,327335.56,79.40
...,...,...,...,...,...,...,...,...,...,...,...
355,2048-12-01,356,30,-1527.48,-1499.15,-28.33,0,-322756.10,-221027.20,7071.10,1.72
356,2049-01-01,357,30,-1527.48,-1503.84,-23.64,0,-324259.94,-221050.85,5567.26,1.35
357,2049-02-01,358,30,-1527.48,-1508.54,-18.95,0,-325768.47,-221069.79,4058.73,0.98
358,2049-03-01,359,30,-1527.48,-1513.25,-14.23,0,-327281.72,-221084.02,2545.48,0.62


In [15]:
# Create Equity DataFrame 

equity_df = pd.DataFrame()
equity_df["payment_date"] = amortization_schedule_df["payment_date"]
equity_df["month"] = amortization_schedule_df["month"]
equity_df["year"] = amortization_schedule_df["year"]
equity_df["ending_balance"] = amortization_schedule_df["ending_balance"]
equity_df["payment"] = amortization_schedule_df["payment"]

house_value = []
value = purchase_price
for row, index in equity_df.iterrows():    
    value = round((value * (1 + ((appreciation_rate/100) / 12))), 2)
    house_value.append(value)
    value = value
    
equity_df["home_value"] = house_value
equity_df["home_equity_without_appreciation"] = purchase_price - equity_df["ending_balance"]
equity_df["home_equity_minus_downpayment_closing_costs"] = equity_df["home_equity_without_appreciation"] - downpayment - going_in_costs
equity_df["equity_from_appreciation"] = equity_df["home_value"] - purchase_price

    
equity_df 

,payment_date,month,year,ending_balance,payment,home_value,home_equity_without_appreciation,home_equity_minus_downpayment_closing_costs,equity_from_appreciation
0,2019-05-01,1,1,329331.98,-1527.48,413460.73,82952.02,-9504.78,1176.73
1,2019-06-01,2,1,328835.21,-1527.48,414640.82,83448.79,-9008.01,2356.82
2,2019-07-01,3,1,328336.88,-1527.48,415824.27,83947.12,-8509.68,3540.27
3,2019-08-01,4,1,327837.00,-1527.48,417011.10,84447.00,-8009.80,4727.10
4,2019-09-01,5,1,327335.56,-1527.48,418201.32,84948.44,-7508.36,5917.32
...,...,...,...,...,...,...,...,...,...
355,2048-12-01,356,30,7071.10,-1527.48,1137227.49,405212.90,312756.10,724943.49
356,2049-01-01,357,30,5567.26,-1527.48,1140473.33,406716.74,314259.94,728189.33
357,2049-02-01,358,30,4058.73,-1527.48,1143728.43,408225.27,315768.47,731444.43
358,2049-03-01,359,30,2545.48,-1527.48,1146992.82,409738.52,317281.72,734708.82


In [16]:
# Create Cash Flow DataFrame 

cashflow_df = pd.DataFrame()

cashflow_df["payment_date"] = amortization_schedule_df["payment_date"]
cashflow_df["month"] = amortization_schedule_df["month"]
cashflow_df["year"] = amortization_schedule_df["year"]
cashflow_df["payment"] = amortization_schedule_df["payment"]
cashflow_df["home_value"] = equity_df["home_value"]
cashflow_df["property_taxes"] = cashflow_df["home_value"] * (property_tax_rate / 12)
cashflow_df["monthly_housing_expense_owning"] = -cashflow_df["payment"] + cashflow_df["property_taxes"] + homeowners_insurance_monthly + hoa_fees
# ^^^ Need Mortgage Insurance calculation for monthly housing expense calculation

# Calculation for Total Owning Costs
monthly_cost = 0
total_costs = []
for cost in cashflow_df["monthly_housing_expense_owning"]:        
    monthly_cost += cost
    total_costs.append(round((cash_to_close + monthly_cost), 2))
cashflow_df["total_owning_cost"] = total_costs   
    
cashflow_df["monthly_rent"] = current_housing_payment
# ^^^  Need Monthly Rent Growth Calculations

# Calculation for Total Rent Costs ("current_housing_payment")
monthly_rent = 0
total_rent = []
for rent in cashflow_df["monthly_rent"]:
    monthly_rent += rent 
    total_rent.append(round(monthly_rent, 2))

cashflow_df["total_renting_cost"] = total_rent
cashflow_df["monthly_saving_expense"] = np.around(cashflow_df["monthly_rent"], decimals=2) - np.around(cashflow_df["monthly_housing_expense_owning"], decimals=2)

# Calculation for Total Cash Flow
saved_spent = 0
total_saved_spent = []
for cash_flow in cashflow_df["monthly_saving_expense"]:    
    saved_spent += cash_flow
    total_saved_spent.append(round(saved_spent, 2))

cashflow_df["total_cash_flow"] = total_saved_spent

cashflow_df

,payment_date,month,year,payment,home_value,property_taxes,monthly_housing_expense_owning,total_owning_cost,monthly_rent,total_renting_cost,monthly_saving_expense,total_cash_flow
0,2019-05-01,1,1,-1527.48,413460.73,689.101217,2291.581217,99321.22,2200,2200,-91.58,-91.58
1,2019-06-01,2,1,-1527.48,414640.82,691.068033,2293.548033,101614.77,2200,4400,-93.55,-185.13
2,2019-07-01,3,1,-1527.48,415824.27,693.040450,2295.520450,103910.29,2200,6600,-95.52,-280.65
3,2019-08-01,4,1,-1527.48,417011.10,695.018500,2297.498500,106207.79,2200,8800,-97.50,-378.15
4,2019-09-01,5,1,-1527.48,418201.32,697.002200,2299.482200,108507.27,2200,11000,-99.48,-477.63
...,...,...,...,...,...,...,...,...,...,...,...,...
355,2048-12-01,356,30,-1527.48,1137227.49,1895.379150,3497.859150,1092045.40,2200,783200,-1297.86,-211815.73
356,2049-01-01,357,30,-1527.48,1140473.33,1900.788883,3503.268883,1095548.66,2200,785400,-1303.27,-213119.00
357,2049-02-01,358,30,-1527.48,1143728.43,1906.214050,3508.694050,1099057.36,2200,787600,-1308.69,-214427.69
358,2049-03-01,359,30,-1527.48,1146992.82,1911.654700,3514.134700,1102571.49,2200,789800,-1314.13,-215741.82


In [17]:
# Create Wealth Created DataFrame 

wealth_created_df = pd.DataFrame()
wealth_created_df["payment_date"] = amortization_schedule_df["payment_date"]
wealth_created_df["month"] = amortization_schedule_df["month"]
wealth_created_df["year"] = amortization_schedule_df["year"]
wealth_created_df["monthly_saving_expense"] = cashflow_df["monthly_saving_expense"]
wealth_created_df["total_cash_flow"] = cashflow_df["total_cash_flow"]
wealth_created_df["home_equity_minus_downpayment_closing_costs"] = equity_df["home_equity_minus_downpayment_closing_costs"]
wealth_created_df["equity_from_appreciation"] = equity_df["equity_from_appreciation"]
wealth_created_df["wealth_created"] = wealth_created_df["total_cash_flow"] + wealth_created_df["home_equity_minus_downpayment_closing_costs"] + wealth_created_df["equity_from_appreciation"]

wealth_created_df

,payment_date,month,year,monthly_saving_expense,total_cash_flow,home_equity_minus_downpayment_closing_costs,equity_from_appreciation,wealth_created
0,2019-05-01,1,1,-91.58,-91.58,-9504.78,1176.73,-8419.63
1,2019-06-01,2,1,-93.55,-185.13,-9008.01,2356.82,-6836.32
2,2019-07-01,3,1,-95.52,-280.65,-8509.68,3540.27,-5250.06
3,2019-08-01,4,1,-97.50,-378.15,-8009.80,4727.10,-3660.85
4,2019-09-01,5,1,-99.48,-477.63,-7508.36,5917.32,-2068.67
...,...,...,...,...,...,...,...,...
355,2048-12-01,356,30,-1297.86,-211815.73,312756.10,724943.49,825883.86
356,2049-01-01,357,30,-1303.27,-213119.00,314259.94,728189.33,829330.27
357,2049-02-01,358,30,-1308.69,-214427.69,315768.47,731444.43,832785.21
358,2049-03-01,359,30,-1314.13,-215741.82,317281.72,734708.82,836248.72


In [18]:
# Dictionary to create wealth created JSON
wealth_created_dict = wealth_created_df.to_dict(orient="list")


In [19]:
# Create Rate of Return DataFrame 

rate_of_return_df = pd.DataFrame()

rate_of_return_df["payment_date"] = amortization_schedule_df["payment_date"]
rate_of_return_df["month"] = amortization_schedule_df["month"]
rate_of_return_df["year"] = amortization_schedule_df["year"]
rate_of_return_df["home_value"] = equity_df["home_value"]
rate_of_return_df["wealth_created"] = wealth_created_df["wealth_created"]
rate_of_return_df["money_received_after_selling_property"] = round((rate_of_return_df["wealth_created"] - (rate_of_return_df["home_value"] * agent_commission)), 2)
rate_of_return_df["present_value_benefit_of_owning_vs_renting"] = round((rate_of_return_df["money_received_after_selling_property"] / \
    (np.power((1 + inflation ), (1/12)))), 2)
rate_of_return_df["percent_return_inflation_adjusted"] = round(((rate_of_return_df["present_value_benefit_of_owning_vs_renting"] / cash_to_close) * 100), 2)

# Calculation for = Annualized Rate of Return 
periods = list(range(1, (len(rate_of_return_df["percent_return_inflation_adjusted"]) + 1)))
rate_of_return_df["annualized_rate_of_return_inflation_adjusted"] = round(((rate_of_return_df["percent_return_inflation_adjusted"] / (periods)) * 12), 2)

rate_of_return_df

,payment_date,month,year,home_value,wealth_created,money_received_after_selling_property,present_value_benefit_of_owning_vs_renting,percent_return_inflation_adjusted,annualized_rate_of_return_inflation_adjusted
0,2019-05-01,1,1,413460.73,-8419.63,-29092.67,-29044.70,-29.93,-359.16
1,2019-06-01,2,1,414640.82,-6836.32,-27568.36,-27522.90,-28.37,-170.22
2,2019-07-01,3,1,415824.27,-5250.06,-26041.27,-25998.33,-26.79,-107.16
3,2019-08-01,4,1,417011.10,-3660.85,-24511.41,-24470.99,-25.22,-75.66
4,2019-09-01,5,1,418201.32,-2068.67,-22978.74,-22940.85,-23.64,-56.74
...,...,...,...,...,...,...,...,...,...
355,2048-12-01,356,30,1137227.49,825883.86,769022.49,767754.48,791.26,26.67
356,2049-01-01,357,30,1140473.33,829330.27,772306.60,771033.18,794.64,26.71
357,2049-02-01,358,30,1143728.43,832785.21,775598.79,774319.94,798.02,26.75
358,2049-03-01,359,30,1146992.82,836248.72,778899.08,777614.79,801.42,26.79


In [20]:
# Create 10-Yr Forecast 

ten_years_df = pd.DataFrame()
ten_years_df["month"] = amortization_schedule_df["month"].loc[0 : 120 - 1]
ten_years_df["year"] = amortization_schedule_df["year"].loc[0 : 120 - 1]
ten_years_df["home_appreciation"] = equity_df["home_value"].loc[0 : 120 - 1]
ten_years_df["equity_from_appreciation"] = equity_df["equity_from_appreciation"].loc[0 : 120 - 1]
ten_years_df["wealth_created"] = wealth_created_df["wealth_created"].loc[0 : 120 - 1]
ten_years_df["percent_return_inflation_adjusted"] = rate_of_return_df["percent_return_inflation_adjusted"].loc[0 : 120 - 1]
ten_years_df["annualized_rate_of_return_inflation_adjusted"] = rate_of_return_df["annualized_rate_of_return_inflation_adjusted"].loc[0 : 120 - 1]
ten_years_df["total_owning_cost"] = cashflow_df["total_owning_cost"].loc[0 : 120 - 1]
ten_years_df["total_renting_cost"] = cashflow_df["total_renting_cost"].loc[0 : 120 - 1]


ten_years_df

,month,year,home_appreciation,equity_from_appreciation,wealth_created,percent_return_inflation_adjusted,annualized_rate_of_return_inflation_adjusted,total_owning_cost,total_renting_cost
0,1,1,413460.73,1176.73,-8419.63,-29.93,-359.16,99321.22,2200
1,2,1,414640.82,2356.82,-6836.32,-28.37,-170.22,101614.77,4400
2,3,1,415824.27,3540.27,-5250.06,-26.79,-107.16,103910.29,6600
3,4,1,417011.10,4727.10,-3660.85,-25.22,-75.66,106207.79,8800
4,5,1,418201.32,5917.32,-2068.67,-23.64,-56.74,108507.27,11000
...,...,...,...,...,...,...,...,...,...
115,116,10,573824.84,161540.84,195363.79,171.49,17.74,377516.97,255200
116,117,10,575462.63,163178.63,197351.19,173.45,17.79,380078.55,257400
117,118,10,577105.10,164821.10,199342.76,175.42,17.84,382642.87,259600
118,119,10,578752.25,166468.25,201338.48,177.39,17.89,385209.94,261800


In [21]:
# Create 10Yr Yearly DateFrame

yearly_df = pd.DataFrame()

ten = list(range(1, 10 +1))

month = []
for x in ten:
     month.append(x * 12)
        
year = []
home_appreciation = []
equity_from_appreciation = []
wealth_created = []
percent_roi = []
annualized_roi = []
total_owning_cost = []
total_renting_cost = []
        
for x in month:
    year.append(ten_years_df["year"].loc[x - 1])
    home_appreciation.append(ten_years_df["home_appreciation"].loc[x - 1])
    equity_from_appreciation.append(ten_years_df["equity_from_appreciation"].loc[x - 1])
    wealth_created.append(ten_years_df["wealth_created"].loc[x - 1])
    percent_roi.append(ten_years_df["percent_return_inflation_adjusted"].loc[x - 1])
    annualized_roi.append(ten_years_df["annualized_rate_of_return_inflation_adjusted"].loc[x - 1])
    total_owning_cost.append(ten_years_df["total_owning_cost"].loc[x - 1])
    total_renting_cost.append(ten_years_df["total_renting_cost"].loc[x - 1])

yearly_df["month"] = month
yearly_df["year"] = year
yearly_df["home_appreciation"] = home_appreciation
yearly_df["equity_from_appreciation"] = equity_from_appreciation
yearly_df["wealth_created"] = wealth_created
yearly_df["percent_return_inflation_adjusted"] = percent_roi
yearly_df["annualized_rate_of_return_inflation_adjusted"] = annualized_roi
yearly_df["total_owning_cost"] = total_owning_cost       
yearly_df["total_renting_cost"] = total_renting_cost


yearly_df

,month,year,home_appreciation,equity_from_appreciation,wealth_created,percent_return_inflation_adjusted,annualized_rate_of_return_inflation_adjusted,total_owning_cost,total_renting_cost
0,12,1,426628.52,14344.52,9160.38,-12.52,-12.52,124659.67,26400
1,24,2,441472.12,29188.12,28758.25,6.88,3.44,152581.96,52800
2,36,3,456832.16,44548.16,48809.56,26.72,8.91,180806.70,79200
3,48,4,472726.63,60442.63,69330.94,47.02,11.76,209344.39,105600
4,60,5,489174.12,76890.12,90339.56,67.79,13.56,238205.93,132000
5,72,6,506193.85,93909.85,111853.24,89.05,14.84,267402.59,158400
6,84,7,523805.76,111521.76,133890.47,110.81,15.83,296946.03,184800
7,96,8,542030.44,129746.44,156470.39,133.11,16.64,326848.31,211200
8,108,9,560889.20,148605.20,179612.83,155.95,17.33,357121.91,237600
9,120,10,580404.11,168120.11,203338.41,179.36,17.94,387779.76,264000


In [22]:
# Dictionary to create ten year JSON
ten_years_dict = ten_years_df.to_dict(orient="list")


In [23]:
# # MatPlotLib Chart

# # create random data
# x_axis = amortization_schedule_df["Payment Date"]


# # create some y data points
# p = -amortization_schedule_df["Cumulative Principal"]
# i = -amortization_schedule_df["Cumulative Interest"]
# b = amortization_schedule_df["Ending Balance"]

# # Create a handle for each plot
# p, = plt.plot(x_axis, p, marker=".",color="blue", label="P")
# i, = plt.plot(x_axis, i, marker=".", color="Red", label="I")
# b, = plt.plot(x_axis, b, marker=".", color="Orange", label="B")


# # Set our legend to where the chart thinks is best
# plt.legend(handles=[p, i, b], loc="best")

# # Create labels for the X and Y axis
# plt.xlabel("Date")
# plt.ylabel("Amortization ($)")


# # display the plot
# plt.show()

In [24]:
# print(amortization_schedule_df.to_string())